In [1]:
import os

In [2]:
!pwd
os.chdir('../')
!pwd

/Users/Bingumalla Likith/Desktop/MLOPS/Project-2/research
/Users/Bingumalla Likith/Desktop/MLOPS/Project-2


In [3]:
import pandas as pd

data = pd.read_csv('artifacts/data_ingestion/taxi_trip_pricing.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Trip_Distance_km       950 non-null    float64
 1   Time_of_Day            950 non-null    object 
 2   Day_of_Week            950 non-null    object 
 3   Passenger_Count        950 non-null    float64
 4   Traffic_Conditions     950 non-null    object 
 5   Weather                950 non-null    object 
 6   Base_Fare              950 non-null    float64
 7   Per_Km_Rate            950 non-null    float64
 8   Per_Minute_Rate        950 non-null    float64
 9   Trip_Duration_Minutes  950 non-null    float64
 10  Trip_Price             951 non-null    float64
dtypes: float64(7), object(4)
memory usage: 86.1+ KB


In [4]:
data.head()

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,Weekday,3.0,Low,Clear,3.56,0.80,0.32,53.82,36.2624
1,47.59,Afternoon,Weekday,1.0,High,Clear,NaN,0.62,0.43,40.57,NaN
2,36.87,Evening,Weekend,1.0,High,Clear,2.70,1.21,0.15,37.27,52.9032
3,30.33,Evening,Weekday,4.0,Low,NaN,3.48,0.51,0.15,116.81,36.4698
4,NaN,Evening,Weekday,3.0,High,Clear,2.93,0.63,0.32,22.64,15.6180


In [5]:
for column , dtype in zip(data.columns , data.dtypes):
    print(f"{column}: {dtype}")

Trip_Distance_km: float64
Time_of_Day: object
Day_of_Week: object
Passenger_Count: float64
Traffic_Conditions: object
Weather: object
Base_Fare: float64
Per_Km_Rate: float64
Per_Minute_Rate: float64
Trip_Duration_Minutes: float64
Trip_Price: float64


In [6]:
data.isnull().sum()

Trip_Distance_km         50
Time_of_Day              50
Day_of_Week              50
Passenger_Count          50
Traffic_Conditions       50
Weather                  50
Base_Fare                50
Per_Km_Rate              50
Per_Minute_Rate          50
Trip_Duration_Minutes    50
Trip_Price               49
dtype: int64

In [7]:
'''
    Perform Data Transformation.
'''

'\n    Perform Data Transformation.\n'

In [8]:
from dataclasses import dataclass
from src.data_science.constants import *
from src.data_science.utils.common import read_yaml, create_directories
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir: Path
    unzip_data_dir: Path
    STATUS_FILE: Path
    all_schema: dict

class ConfigurationManager:
    def __init__(self,
                config_path = CONFIG_FILE_PATH,
                params_path = PARAMS_FILE_PATH,
                schema_path = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        self.schema = read_yaml(schema_path)
        
        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(**config , all_schema=schema)
        return data_validation_config

In [9]:
from src.data_science import logger
import os

class DataValidation:
    def __init__(self, config:DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try: 
            validation_status = True
            data = pd.read_csv(self.config.unzip_data_dir)
            all_columns = list(data.columns)
            all_datatypes = list(data.dtypes)

            all_schema = self.config.all_schema

            for col, data_type in zip(all_columns, all_datatypes):
                dtype = all_schema.get(col , None)
                if not dtype or dtype != data_type:
                    validation_status = False
                else:
                    validation_status = True
                with open(self.config.STATUS_FILE , 'a') as file:
                    file.write(f"{col} -> Validation Status: {validation_status}\n")

            return validation_status
        except Exception as e:
            raise e

In [10]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(data_validation_config)
    
    data_validation.validate_all_columns()

except Exception as e:
    raise e

[2024-12-29 21:42:59,237 : INFO : common : Yaml file : config/config.yaml loaded successfully !!]
[2024-12-29 21:42:59,269 : INFO : common : Yaml file : params.yaml loaded successfully !!]
[2024-12-29 21:42:59,271 : INFO : common : Yaml file : schema.yaml loaded successfully !!]
[2024-12-29 21:42:59,271 : INFO : common : Created directory at : artifacts]
[2024-12-29 21:42:59,271 : INFO : common : Created directory at : artifacts/data_validation]
